**Load data**

In [ ]:
# Connect to Google drive
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir("drive/My Drive/Colab Notebooks/Source_code/LSTM_data/LSTM_dataset")

Mounted at /content/drive


In [ ]:
# Download library
!pip install contractions
!pip install autocorrect
!pip install tqdm
!pip install emoticon-fix
!pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Import library
import os
import re
import pandas as pd

import contractions
import nltk
import numpy as np
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from autocorrect import Speller
from tqdm import tqdm
from collections import Counter
import emoji
from emoticon_fix import emoticon_fix

# Lemmatize with POS Tag
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet


nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

tqdm.pandas()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
pd.options.display.max_colwidth = 10000

In [ ]:
# Read file mobile dataset for train LSTM as dataframe
df= pd.read_csv('combined_LSTM_dataset.csv')


In [ ]:
df[0:5]

,text,sentiment,rate
0,The camera is very slow :(,neutral,3.0
1,I was happy I got a case and a charger that was in the description but I was told the phone works perfectly fine but it doesn't. The phone twitches which is beyond annoying. I don't appreciate that,negative,1.0
2,"I bought this phone for my wife and I wanted to wait for a month and a half to leave a review because I really wanted her to live with this phone and really utilize it. I have to say, she LOVES IT! I am responsible for the phone purchases for my family. We have T-Mobile and they don't subsidize phones anymore so I needed to replace my wife's older phone with a quality no contract phone and after researching for 3 months, I decided to try BLU. The phone came quickly and it even had the screen protector already installed. I put in the micro sim card and we were off and running! I will admit after using the phone for a hour straight to get all apps loaded and other settings fine tuned, the phone back got very warm! I was a little surprised however if cooled down once we were done and let the phone sit. The screen is awesome! Everything installed with no issues. I had my wife let me know if she had any issues and after a month and a half, the only complaint she has had is the phone back gets warm when she uses it to play her games. She will sit for few hours after dinner and play her games on her phone. When she does, the phone gets very warm. From my research it looks like this is common with a ton of other phones when you use it to play apps hours on end. I am not familiar with this as i don't play games on my phone and never had any heat issues with my Nexus 5. Back to the BLU, the only other complaint my wife had is the camera. The phone takes very nice pictures however you must stay very still and the lag between snapping pictures is torturous. You have to stand still and have your subject stay still, even after you press the photo button. I think this is my be a software issue and maybe BLU will come out with an update. Other than the heat issue and the camera, there are no other issues. The phone came with a case, can't beat that! The battery lasts all day, needs to be recharged each night. Apps open quick and videos play with no lag. I still gave the phone 5 stars because I am not sold the heat issue is only BLU and the camera takes really nice photos even with the lag and the bottom line is my wife got a lot of phone for the price. The LTE works fine with T-Mobile. No issues there! I will buy additional phones from BLU as more of my family members are in need of an upgraded phone. I just ordered a BLU Studio LTE for my youngest son. I will review that one after living with it for a month.",positive,5.0
3,HTC One in White is good; in black it is better. Love the new phone. I come from Samsung bandwagon...Thanks HTC.,positive,5.0
4,"This phone is definitely one of the fastest on the market at the moment. I have dropped it several times on hard floors, and there is no visible damage at all. I have also handled it with wet hands, and it still works flawlessly. Although, the fingerprint scanner doesn't get along with water drips. The edge screen is a great addition, and I have gotten use out of the news feature when the phone is locked. The battery life is great, since it lasts about a day with moderate to heavy use.Also, for those who are wondering, this model supports most LTE bands, and it is also supported by most GSM carriers. I use it with Metro PCS, and it works perfectly.",positive,5.0


**Remove noise**

In [ ]:
# Delete all html tags and urls in text 
def html_remover(text):
    beauti = BeautifulSoup(text, 'html.parser')
    return beauti.get_text()

def web_associated(text):
    text = str(text)
    text = html_remover(text)
    text = re.sub(r'(www\S+ ?)|(\w+\.com)|(http\S+)', '', text)
    return text

# Delete noise charactors
def remove_noise(text):
  return re.sub(r'(\'s|s\'|’s|s’|\(|\)|\[|\]|\{|\"|\'|\“|\’|\”|\‘|(\'.*))|\`|\—', '', text)

In [ ]:
# Delete noise processing

spell = Speller(fast=True)
def noise_preprocessing(text):
  # delete all html tags and urls in text 
  text = web_associated(text) 

  # expand contractions
  text = contractions.fix(text)

  # remove noise charactor
  text = remove_noise(text)

  # convert emoticon to description word for this emoticon
  text = emoticon_fix.emoticon_fix(text)

  # convert emoji to description word for this emoji
  text = emoji.demojize(text).replace("_"," ").replace(':',' ')

  # Delete charactor repeate more than 2 times in a word
  # Ex: Helllllllo -> Hello
  text = re.sub(r'(.)\1+', r'\1\1', text)

  # Correct wrong word
  text = spell(text)

  # lowercase word
  text = text.lower()
  return text


In [ ]:
# Tokenize text
def tokenize(text):
  return word_tokenize(text)

# Create punctuations list for all punctuation will be deleted after tokenize
punctuations = [token for token in '?!.,"#$%\'()*+-/:;<=>@[\\]^_`{|}~“”’‘&']
punctuations.extend(['...','``','--','—','——','•','«','……','––','—-','°','¬','·','▪','–-','—','−','―','›','-','..','»','+','*','‹','♦','₪','%N%','%$%','%^%','「','」','『','』','〈','〉','》','《','¿'])

# Remove noise puctuations and word has non ascii characters
def remove_punc(tokens):
    return [token for token in tokens if token not in punctuations and token.isascii()]

# Remove token has only one charactor
def remove_one_character(tokens):
    return [token for token in tokens if len(token)>1]


In [ ]:
# Remove stopwords

# Modified nltk stopwords list
stop_words = stopwords.words('english')

# get all negative words from stopwords
negative_words = []
for word in stop_words:
  if word.endswith("n't"):
    negative_words.append(word)
    negative_words.append(word[:-2])
negative_words.extend(["not",'ain','no','nor'])


# remove all negative words from stopwords
stop_words = [word for word in stop_words if word not in negative_words]
model_words = ['can','could','may','might','will','would','must','shall','should','have','has','had','maybe']

#filter word pauses as 'okay','hi',
filter_word_pauses = ['um','uhm','um-hum','uh','yeah','uh-huh','oh','huh','uh-huh']
stop_words.extend(filter_word_pauses)
stop_words.extend(model_words)


stop_words_dict = Counter(stop_words)
#clean stopwords
def stopword(tokens):
    return [token for token in tokens if token not in stop_words_dict]
    

**Normalize text**

In [ ]:
# Normalization preprocessing
def normalization_preprocessing(data):
    tokens = tokenize(data)
    tokens = remove_punc(tokens)
    tokens = remove_one_character(tokens)
    tokens = stopword(tokens)
    return tokens

**LSTM preprocessing**

In [ ]:
# LSTM Preprocessing 
def preprocessing(text):      
    text = noise_preprocessing(text)   
    tokens = normalization_preprocessing(text)
    return tokens

In [ ]:
#Concate tokens to text
def concate_text(tokens):
  temp_tokens = []
  if type(tokens)==list:
    temp_tokens.extend(tokens)
  elif type(tokens) == str:
    temp_tokens.append(tokens)
  return " ".join(temp_tokens)

**Execute preprocessing**

In [ ]:
# Run preprocessing and save result tokens to tokens columns of df
df['tokens'] = df.text.progress_apply(lambda x: preprocessing(x))

100%|██████████| 90000/90000 [03:25<00:00, 437.50it/s]


In [ ]:
df[0:5].tokens

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                [camera, slow]
1                                                                                                                                                                                                                                                       

In [ ]:
# Execute concate tokens
df['clean_text'] = df.tokens.progress_apply(lambda x: concate_text(x))

100%|██████████| 90000/90000 [00:00<00:00, 377072.20it/s]


In [ ]:
df[0:5].clean_text

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [ ]:
#Save to Google drive
df.to_csv('preprocessed_LSTM_dataset.csv', encoding='utf-8', index=False)